<a href="https://colab.research.google.com/github/falahamro/twitter_harvest_academic/blob/main/Twitter_Query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv
import argparse
import json
import logging
=import os.path
import stat
import re
import requests
import shlex
import subprocess
import sys
import time
import csv
from datetime import timedelta
import os
import json
import requests
import tweepy
import dateutil.parser
import pandas as pd
import seaborn as sns
import numpy as np

In [ ]:
client = tweepy.Client(bearer_token = 'AAAAAAAAAAAAAAAAAAAAAH0CYgEAAAAA')
# replace with your own search query (replace MAGA with anything else)
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers
os.environ['TOKEN'] = ''


In [ ]:
tweets = []
for response in tweepy.Paginator(client.search_all_tweets, 
                                 query = 'stopthesteal -is:retweet lang:en',
                                 user_fields = ['username', 'public_metrics','entities', 'description', 'location', 'verified'],
                                 tweet_fields = ['id','text','author_id','in_reply_to_user_id','geo','conversation_id','created_at,lang','public_metrics','referenced_tweets','reply_settings',],
                                 expansions = 'author_id,in_reply_to_user_id,geo.place_id,entities.mentions.username,referenced_tweets.id',
                                 place_fields = ['full_name','id','country','country_code','geo','name','place_type',],
                                 start_time = '2022-01-20T00:00:00Z',
                                 end_time = '2022-04-19T00:00:00Z',
                              max_results=500):
    time.sleep(1)
    tweets.append(response)
result = []
user_dict = {}
# Loop through each response object
for response in tweets:
    # Take all of the users, and put them into a dictionary of dictionaries with the info we want to keep
    for user in response.includes['users']:
        user_dict[user.id] = {'username': user.username, 
                              'listed_count': user.public_metrics['listed_count'],
                              'followers': user.public_metrics['followers_count'],
                              'following': user.public_metrics['following_count'],
                              'listed_count': user.public_metrics['listed_count'],

                              'tweets': user.public_metrics['tweet_count'],
                             #'entities': user.public_metrics['entities'],

                              #'description': user.description,
                              #'location': user.location,
                              'verified': user.verified,

                             }
    for tweet in response.data: 
        # For each tweet, find the author's information
        author_info = user_dict[tweet.author_id]
        # Put all of the information we want to keep in a single dictionary for each tweet
        result.append({'author_id': tweet.author_id, 
                       
                       #'geo': tweet.geo, 
                      #'referenced_tweets': tweet.referenced_tweets, 
                       #'username': author_info['username'],

                       'author_followers': author_info['followers'],
                       'author_followee': author_info['following'],
                       'listed_count': author_info['listed_count'],
                       'author_tweets': author_info['tweets'],
                       #'author_description': author_info['description'],
                       #'author_location': author_info['location'],
                       ##author_verified': author_info['verified'],
                       'text': tweet.text,
                       'created_at': tweet.created_at,
                       #'reply_settings': tweet.reply_settings,
                       #'in_reply_to_user_id': tweet.in_reply_to_user_id,
                       'retweets': tweet.public_metrics['retweet_count'],
                       'replies': tweet.public_metrics['reply_count'],
                       'likes': tweet.public_metrics['like_count'],

                       'quote_count': tweet.public_metrics['quote_count'],
                        #'favourites_count': tweet.public_metrics['favourites_count']


                      })
import math

  
import pandas as pd
df = pd.DataFrame(result)
df.to_csv('stopthesteal.csv', index=False)